In [1]:
import os

from collections import OrderedDict

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import scipy

import warnings

from Standard_functions import plot_features, get_user_split_data, plot_fit_score_pred

warnings.filterwarnings('always') 
np.seterr(divide='ignore', invalid='ignore')
%matplotlib inline

In [2]:
os.chdir('../Data/')

# CREATING TEST SET

In [3]:
ord_prod_prior_df = pd.read_csv('order_products__prior.csv')
ord_df = pd.read_csv('orders.csv')
ord_prod_prior_df = ord_prod_prior_df.merge(ord_df, on='order_id')
prod_df = pd.read_csv('products.csv')
dep_df = pd.read_csv('departments.csv')

In [4]:
df = (ord_prod_prior_df.groupby(['product_id','user_id'],as_index=False) 
                              .agg({'order_id':'count'}) 
                              .rename(columns={'order_id':'num_ord_per_user_per_prod'}))

Moving split to after feature engineering

```python 
test_ids = ord_df[ord_df.eval_set=='test'].user_id  
df = df[df.user_id.isin(test_ids)]
```

## Feature Engineering

Our feature engineering began in the 'Organizing data section.  
We will visualize that first feature below and begin creating new features.

As you can see, we're setting the bar pretty low.

## Product Features
Note that product features are not user-specific.

### `product_total_orders`
Total number of orders placed for a given product.
### `product_avg_add_to_cart_order`
Average order in which a product was add to a users cart.

In [5]:
prod_features = ['product_total_orders','product_avg_add_to_cart_order']

df_prod_features = (ord_prod_prior_df.groupby(['product_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'),
                                                    ('add_to_cart_order','mean')])))
df_prod_features.columns = ['product_id'] + prod_features
df = df.merge(df_prod_features, on='product_id')

`product_total_orders` had some pretty enourmous rightward skew that ultimately resulted in the model producing 0 positive predictions. As a result I took the log base 2 transformation to punish outliers.  
Note that I attempted a log base 10 transformation but log base 2 had better results.

In [6]:
df.product_total_orders = np.log2(df.product_total_orders)

## User Features
`'user_total_orders','user_avg_cartsize','user_total_products','user_avg_days_since_prior_order'`

In [7]:
user_features = ['user_total_orders','user_avg_cartsize',
                 'user_total_products','user_avg_days_since_prior_order']

df_user_features = (ord_prod_prior_df.groupby(['user_id'],as_index=False)
                    .agg(OrderedDict(
                       [('order_id',
                         ['nunique', # user_total_orders
                        (lambda x: x.shape[0] / x.nunique())]), # user_avg_cartsize
                        ('product_id','nunique'), # user_total_products
                        ('days_since_prior_order','mean')]))) # user_avg_days_since_prior_order
df_user_features.columns = ['user_id'] + user_features
df = df.merge(df_user_features, on='user_id')

## User-Product features

In [8]:
user_prod_features = ['user_product_avg_add_to_cart_order']

df_user_prod_features = (ord_prod_prior_df.groupby(
                        ['product_id','user_id'],as_index=False) \
                        .agg(OrderedDict(
                        [('add_to_cart_order','mean')])))

df_user_prod_features.columns = ['product_id','user_id'] + user_prod_features 
df = df.merge(df_user_prod_features,on=['user_id','product_id'])
df['user_product_order_freq'] = df['num_ord_per_user_per_prod'] / df['user_total_orders'] 

## Number of Orders Placed Since a Product was Last Ordered
## Percentage of Orders Containing a Product

In [9]:
new_feats = ['total_orders_containig_product','most_recent_order_by_user_by_product']

most_recent_order_by_user_by_product = (ord_prod_prior_df.groupby(['user_id','product_id'],
                                                        as_index=False).agg(OrderedDict(
                                                        [('reordered','sum'),
                                                        ('order_number','max')])))
most_recent_order_by_user_by_product.columns = ['user_id','product_id'] + new_feats

max_orders_per_user = ord_prod_prior_df.groupby(['user_id'],as_index=False).order_number.max()
max_orders_per_user.columns = ['user_id'] + ['number_orders_since_product_last_ordered']

number_orders_since_product_last_ordered = pd.merge(most_recent_order_by_user_by_product, 
                                                    max_orders_per_user,on='user_id')

number_orders_since_product_last_ordered['percent_of_time_product_ordered'] =(
    number_orders_since_product_last_ordered['total_orders_containig_product'] / 
    number_orders_since_product_last_ordered['number_orders_since_product_last_ordered'])
    # Note that we use the column name `number_orders_since_product_last_ordered` as that
    # is currently set to the max max order and does not yet actually contain the 
    # number of orders since a product was last ordered.

percent_of_time_product_ordered = number_orders_since_product_last_ordered[['percent_of_time_product_ordered',
                                                                      'user_id','product_id']]

number_orders_since_product_last_ordered = (number_orders_since_product_last_ordered[
                    ['most_recent_order_by_user_by_product','number_orders_since_product_last_ordered']]
                    .diff(axis=1).drop('most_recent_order_by_user_by_product',axis=1))

most_recent_order_by_user_by_product.drop('most_recent_order_by_user_by_product',
                                          axis=1,inplace=True) # we're just using this to get back our product_id and user_id
number_orders_since_product_last_ordered = pd.merge(number_orders_since_product_last_ordered,
                            most_recent_order_by_user_by_product,left_index=True,right_index=True)

combined_feats = pd.merge(number_orders_since_product_last_ordered,percent_of_time_product_ordered,
                        on=['user_id','product_id'])

combined_feats.drop('total_orders_containig_product',axis=1,inplace=True)

df = pd.merge(df, combined_feats,on=['user_id','product_id'])

## Time since most recent order by user

In [10]:
mean_hod = (ord_df.groupby(['user_id'],as_index=False)
                    .agg({'order_hour_of_day':'mean'}))
mean_hod.columns = ['user_id','mean_order_hour_of_day_by_user']

most_recent_order_time = (ord_df.groupby(['user_id'],as_index=False)
                    .agg({'order_number':'max'}))
temp = pd.merge(most_recent_order_time,ord_df, on=['user_id','order_number'],how='left')
temp = pd.merge(temp[['user_id','order_hour_of_day']], mean_hod, on=['user_id'])
temp['diff_between_average_and_current_order_time'] = (temp['order_hour_of_day'] - 
                                        temp['mean_order_hour_of_day_by_user'])
df = pd.merge(df, temp.drop('order_hour_of_day',axis=1)
              ,on=['user_id'])

## Weighting products for more of more recent orders

In [11]:
recent_order_weight = ord_prod_prior_df.loc[:,['product_id', 'reordered', 'user_id', 'order_number']]
max_order_number = ord_df.groupby(['user_id'],as_index=False).agg({'order_number':'max'})
max_order_number.columns = ['user_id','max_order_number']
recent_order_weight = recent_order_weight.merge(max_order_number,on='user_id')
# Here we set max_order_number to be the nth traingular number of the max order number
recent_order_weight['max_order_number'] = ((recent_order_weight.max_order_number *
                                            (recent_order_weight.max_order_number + 1)) / 2)
recent_order_weight['recent_order_weight'] = ((recent_order_weight.reordered 
                                              * recent_order_weight.order_number)
                                              / recent_order_weight.max_order_number)
recent_order_weight = recent_order_weight.groupby(['product_id','user_id'],as_index=False).sum()
recent_order_weight.drop(['reordered','order_number'],axis=1,inplace=True)
df = pd.merge(df, recent_order_weight,on=['user_id','product_id'])

### Removing Max Order Number
Max order number clearly had a significant negative impact and has been removed.

In [12]:
df.drop(['max_order_number'],axis=1,inplace=True)

## Adding Department Dummy Variable

In [13]:
department_name_product_id = (prod_df.merge(dep_df,on='department_id')
                              .drop(['product_name','aisle_id','department_id'],axis=1))
dep_dummies = pd.get_dummies(department_name_product_id,drop_first=False)
dummies_cols = dep_dummies.columns.tolist()
new_cols = []
for name in dummies_cols:
    new_cols.append(str.replace(name,' ','_'))
dep_dummies.columns = new_cols

dep_dummies['positvely_corr_departments'] = dep_dummies.department_dairy_eggs + dep_dummies.department_produce
dep_dummies['negatively_corr_departments'] = dep_dummies.department_pantry + dep_dummies.department_personal_care
dep_dummies = dep_dummies[['negatively_corr_departments','positvely_corr_departments','product_id']]

df = pd.merge(df, dep_dummies,on=['product_id'])

In [14]:
test_ids = ord_df[ord_df.eval_set=='test'].user_id
df = df[df.user_id.isin(test_ids)]

In [15]:
df.reset_index(inplace=True,drop=True)

In [16]:
pd.to_pickle(df,'full_features_with_dep_test')

In [17]:
df.drop(['negatively_corr_departments','positvely_corr_departments'],axis=1,inplace=True)

In [18]:
pd.to_pickle(df,'full_features_test')

# TRAINING SET

In [3]:
ord_prod_prior_df = pd.read_csv('order_products__prior.csv')
ord_prod_train_df = pd.read_csv('order_products__train.csv')
ord_df = pd.read_csv('orders.csv')
ord_prod_prior_df = ord_prod_prior_df.merge(ord_df, on='order_id')
ord_prod_train_df = ord_prod_train_df.merge(ord_df, on='order_id')
prod_df = pd.read_csv('products.csv')
dep_df = pd.read_csv('departments.csv')

In [4]:
df = (ord_prod_prior_df.groupby(['product_id','user_id'],as_index=False) 
                              .agg({'order_id':'count'}) 
                              .rename(columns={'order_id':'num_ord_per_user_per_prod'}))

test_ids = ord_df[ord_df.eval_set=='test'].user_id  
df = df[~df.user_id.isin(test_ids)]

In [5]:
test_carts = (ord_prod_train_df.groupby('user_id',as_index=False)
                                      .agg({'product_id':(lambda x: set(x))})
                                      .rename(columns={'product_id':'latest_cart'}))

df = df.merge(test_carts, on='user_id')
df['in_cart'] = (df.apply(lambda row: row['product_id'] in row['latest_cart'], axis=1).astype(int))

In [6]:
df.head()

,product_id,user_id,num_ord_per_user_per_prod,latest_cart,in_cart
0,1,138,2,{42475},0
1,907,138,2,{42475},0
2,1000,138,1,{42475},0
3,3265,138,1,{42475},0
4,4913,138,1,{42475},0


## Feature Engineering

Our feature engineering began in the 'Organizing data section.  
We will visualize that first feature below and begin creating new features.

As you can see, we're setting the bar pretty low.

## Product Features
Note that product features are not user-specific.

### `product_total_orders`
Total number of orders placed for a given product.
### `product_avg_add_to_cart_order`
Average order in which a product was add to a users cart.

In [7]:
prod_features = ['product_total_orders','product_avg_add_to_cart_order']

df_prod_features = (ord_prod_prior_df.groupby(['product_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'),
                                                    ('add_to_cart_order','mean')])))
df_prod_features.columns = ['product_id'] + prod_features
df = df.merge(df_prod_features, on='product_id')

`product_total_orders` had some pretty enourmous rightward skew that ultimately resulted in the model producing 0 positive predictions. As a result I took the log base 2 transformation to punish outliers.  
Note that I attempted a log base 10 transformation but log base 2 had better results.

In [8]:
df.product_total_orders = np.log2(df.product_total_orders)

## User Features
`'user_total_orders','user_avg_cartsize','user_total_products','user_avg_days_since_prior_order'`

In [9]:
user_features = ['user_total_orders','user_avg_cartsize',
                 'user_total_products','user_avg_days_since_prior_order']

df_user_features = (ord_prod_prior_df.groupby(['user_id'],as_index=False)
                    .agg(OrderedDict(
                       [('order_id',
                         ['nunique', # user_total_orders
                        (lambda x: x.shape[0] / x.nunique())]), # user_avg_cartsize
                        ('product_id','nunique'), # user_total_products
                        ('days_since_prior_order','mean')]))) # user_avg_days_since_prior_order
df_user_features.columns = ['user_id'] + user_features
df = df.merge(df_user_features, on='user_id')

## User-Product features

In [10]:
user_prod_features = ['user_product_avg_add_to_cart_order']

df_user_prod_features = (ord_prod_prior_df.groupby(
                        ['product_id','user_id'],as_index=False) \
                        .agg(OrderedDict(
                        [('add_to_cart_order','mean')])))

df_user_prod_features.columns = ['product_id','user_id'] + user_prod_features 
df = df.merge(df_user_prod_features,on=['user_id','product_id'])
df['user_product_order_freq'] = df['num_ord_per_user_per_prod'] / df['user_total_orders'] 

## Number of Orders Placed Since a Product was Last Ordered
## Percentage of Orders Containing a Product

In [11]:
new_feats = ['total_orders_containig_product','most_recent_order_by_user_by_product']

most_recent_order_by_user_by_product = (ord_prod_prior_df.groupby(['user_id','product_id'],
                                                        as_index=False).agg(OrderedDict(
                                                        [('reordered','sum'),
                                                        ('order_number','max')])))
most_recent_order_by_user_by_product.columns = ['user_id','product_id'] + new_feats

max_orders_per_user = ord_prod_prior_df.groupby(['user_id'],as_index=False).order_number.max()
max_orders_per_user.columns = ['user_id'] + ['number_orders_since_product_last_ordered']

number_orders_since_product_last_ordered = pd.merge(most_recent_order_by_user_by_product, 
                                                    max_orders_per_user,on='user_id')

number_orders_since_product_last_ordered['percent_of_time_product_ordered'] =(
    number_orders_since_product_last_ordered['total_orders_containig_product'] / 
    number_orders_since_product_last_ordered['number_orders_since_product_last_ordered'])
    # Note that we use the column name `number_orders_since_product_last_ordered` as that
    # is currently set to the max max order and does not yet actually contain the 
    # number of orders since a product was last ordered.

percent_of_time_product_ordered = number_orders_since_product_last_ordered[['percent_of_time_product_ordered',
                                                                      'user_id','product_id']]

number_orders_since_product_last_ordered = (number_orders_since_product_last_ordered[
                    ['most_recent_order_by_user_by_product','number_orders_since_product_last_ordered']]
                    .diff(axis=1).drop('most_recent_order_by_user_by_product',axis=1))

most_recent_order_by_user_by_product.drop('most_recent_order_by_user_by_product',
                                          axis=1,inplace=True) # we're just using this to get back our product_id and user_id
number_orders_since_product_last_ordered = pd.merge(number_orders_since_product_last_ordered,most_recent_order_by_user_by_product,
                        left_index=True,right_index=True)

combined_feats = pd.merge(number_orders_since_product_last_ordered,percent_of_time_product_ordered,
                        on=['user_id','product_id'])

combined_feats.drop('total_orders_containig_product',axis=1,inplace=True)

df = pd.merge(df, combined_feats,on=['user_id','product_id'])

## Time since most recent order by user

In [12]:
mean_hod = (ord_df.groupby(['user_id'],as_index=False)
                    .agg({'order_hour_of_day':'mean'}))
mean_hod.columns = ['user_id','mean_order_hour_of_day_by_user']

most_recent_order_time = (ord_df.groupby(['user_id'],as_index=False)
                    .agg({'order_number':'max'}))
temp = pd.merge(most_recent_order_time,ord_df, on=['user_id','order_number'],how='left')
temp = pd.merge(temp[['user_id','order_hour_of_day']], mean_hod, on=['user_id'])
temp['diff_between_average_and_current_order_time'] = (temp['order_hour_of_day'] - 
                                        temp['mean_order_hour_of_day_by_user'])
df = pd.merge(df, temp.drop('order_hour_of_day',axis=1)
              ,on=['user_id'])

## Weighting products for more of more recent orders

In [13]:
recent_order_weight = ord_prod_prior_df.loc[:,['product_id', 'reordered', 'user_id', 'order_number']]
max_order_number = ord_df.groupby(['user_id'],as_index=False).agg({'order_number':'max'})
max_order_number.columns = ['user_id','max_order_number']
recent_order_weight = recent_order_weight.merge(max_order_number,on='user_id')
# Here we set max_order_number to be the nth traingular number of the max order number
recent_order_weight['max_order_number'] = ((recent_order_weight.max_order_number *
                                            (recent_order_weight.max_order_number + 1)) / 2)
recent_order_weight['recent_order_weight'] = ((recent_order_weight.reordered 
                                              * recent_order_weight.order_number)
                                              / recent_order_weight.max_order_number)
recent_order_weight = recent_order_weight.groupby(['product_id','user_id'],as_index=False).sum()
recent_order_weight.drop(['reordered','order_number'],axis=1,inplace=True)
df = pd.merge(df, recent_order_weight,on=['user_id','product_id'])

### Removing Max Order Number
Max order number clearly had a significant negative impact and has been removed.

In [14]:
df.drop(['max_order_number'],axis=1,inplace=True)

In [15]:
pd.to_pickle(df,'full_features_train')

## Adding Department Dummy Variable

In [16]:
department_name_product_id = (prod_df.merge(dep_df,on='department_id')
                              .drop(['product_name','aisle_id','department_id'],axis=1))
dep_dummies = pd.get_dummies(department_name_product_id,drop_first=False)
dummies_cols = dep_dummies.columns.tolist()
new_cols = []
for name in dummies_cols:
    new_cols.append(str.replace(name,' ','_'))
dep_dummies.columns = new_cols

dep_dummies['positvely_corr_departments'] = dep_dummies.department_dairy_eggs + dep_dummies.department_produce
dep_dummies['negatively_corr_departments'] = dep_dummies.department_pantry + dep_dummies.department_personal_care
dep_dummies = dep_dummies[['negatively_corr_departments','positvely_corr_departments','product_id']]

df = pd.merge(df, dep_dummies,on=['product_id'])

In [17]:
pd.to_pickle(df,'full_features_with_dep_train')